In [1]:
import requests
import pandas as pd

In [2]:
ruta_archivo = r"../data/processed/datos_filtradosv1.csv"
# Cargamos los datos del CSV a un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

C:\Users\caste\AppData\Local\Temp\ipykernel_11780\1893727086.py:3: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)


In [3]:
# Seleccionamos las columnas que queremos mantener
df = df[['Departamento',
         'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
         'Estado Contrato',
         'Tipo de Contrato',
         'Modalidad de Contratacion',
         'Fecha de Firma', 'Fecha de Inicio del Contrato',
         'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
         'Fecha de Fin de Ejecucion',
         'Es Grupo', 'Es Pyme', 'Liquidación',
         'Obligación Ambiental',
         'Valor del Contrato', 'Valor Facturado',
         'Valor Pendiente de Pago', 'Valor Pagado',
         'Valor Pendiente de Ejecucion',
         'Saldo CDP',
         'EsPostConflicto', 'Destino Gasto',
         'Origen de los Recursos', 'Dias Adicionados',
         'Género Representante Legal',
         'Presupuesto General de la Nacion – PGN',
         'Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)',
         'Recursos Propios'
         ]]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2689335 entries, 0 to 2689334
Data columns (total 31 columns):
 #   Column                                                              Dtype  
---  ------                                                              -----  
 0   Departamento                                                        object 
 1   Orden                                                               object 
 2   Sector                                                              object 
 3   Rama                                                                object 
 4   Entidad Centralizada                                                object 
 5   Estado Contrato                                                     object 
 6   Tipo de Contrato                                                    object 
 7   Modalidad de Contratacion                                           object 
 8   Fecha de Firma                                                      obje

#### Conversion de las fechas a DateTime

In [4]:
columnas_fecha = ['Fecha de Firma', 'Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato',
                  'Fecha de Inicio de Ejecucion', 'Fecha de Fin de Ejecucion']

# Iterar sobre las columnas y convertirlas al tipo de dato 'date'
for columna in columnas_fecha:
    df[columna] = pd.to_datetime(df[columna], format='%m/%d/%Y', errors='coerce')
    df[columna] = pd.to_datetime(df[columna], format='%Y-%m-%d %H:%M:%S', errors='coerce')

#### Filtrar por el estado del contrato

In [5]:
estados_contrato = ["terminado", "Cerrado", "cedido", "Prorrogado", "Suspendido"]

# Filtrar las filas del dataframe basándote en la columna "Estado Contrato"
df_filtradoNoEjecucion = df[df["Estado Contrato"].isin(estados_contrato)]
df_filtradoNoEjecucion

,Departamento,Orden,Sector,Rama,Entidad Centralizada,Estado Contrato,Tipo de Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,...,Valor Pendiente de Ejecucion,Saldo CDP,EsPostConflicto,Destino Gasto,Origen de los Recursos,Dias Adicionados,Género Representante Legal,Presupuesto General de la Nacion – PGN,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios
0,Santander,Territorial,Servicio Público,Ejecutivo,Centralizada,terminado,Prestación de servicios,Contratación directa,2021-08-17,2021-08-19,...,0.0,13750000,No,Inversión,Distribuido,0,Hombre,0,13750000,0
2,Atlántico,Nacional,Educación Nacional,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2020-02-07,2020-02-07,...,0.0,366080000,No,Inversión,Distribuido,0,No Definido,0,0,0
3,Distrito Capital de Bogotá,Nacional,Hacienda y Crédito Público,Ejecutivo,No Definido,Cerrado,Prestación de servicios,Contratación directa,2019-04-15,2019-02-20,...,1557298.0,66379827,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
9,Distrito Capital de Bogotá,Nacional,agricultura,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2021-01-20,2021-01-21,...,1744200.0,1034967000,No,Inversión,Distribuido,0,Mujer,22032000,0,0
10,Distrito Capital de Bogotá,Nacional,Educación Nacional,Corporación Autónoma,Descentralizada,terminado,Prestación de servicios,Contratación régimen especial,2021-08-05,2021-08-09,...,2640000.0,52800000,No,Funcionamiento,Recursos Propios,0,No Definido,0,0,49500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689323,Cundinamarca,Territorial,No aplica/No pertenece,Ejecutivo,Descentralizada,Cerrado,Prestación de servicios,Contratación directa,2018-08-21,2018-08-21,...,20279040.0,20279040,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
2689327,Huila,Nacional,defensa,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Mínima cuantía,2021-04-06,2021-04-14,...,21000000.0,32550000,No,Funcionamiento,Distribuido,0,No Definido,24500000,0,0
2689331,Distrito Capital de Bogotá,Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,terminado,DecreeLaw092/2017,Contratación régimen especial,2022-03-24,2022-03-29,...,6957491.0,58912135,No,Funcionamiento,Recursos Propios,0,Mujer,0,0,6957491
2689332,Atlántico,Nacional,Trabajo,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2023-01-26,2023-02-01,...,14080000.0,2201918000,No,Inversión,Distribuido,0,No Definido,20680000,0,0


#### Filtrar por año del contrato

In [6]:
df_filtradoAnio = df_filtradoNoEjecucion[df_filtradoNoEjecucion['Fecha de Inicio del Contrato'].dt.year >= 2019]
df_filtradoAnio

,Departamento,Orden,Sector,Rama,Entidad Centralizada,Estado Contrato,Tipo de Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,...,Valor Pendiente de Ejecucion,Saldo CDP,EsPostConflicto,Destino Gasto,Origen de los Recursos,Dias Adicionados,Género Representante Legal,Presupuesto General de la Nacion – PGN,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios
0,Santander,Territorial,Servicio Público,Ejecutivo,Centralizada,terminado,Prestación de servicios,Contratación directa,2021-08-17,2021-08-19,...,0.0,13750000,No,Inversión,Distribuido,0,Hombre,0,13750000,0
2,Atlántico,Nacional,Educación Nacional,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2020-02-07,2020-02-07,...,0.0,366080000,No,Inversión,Distribuido,0,No Definido,0,0,0
3,Distrito Capital de Bogotá,Nacional,Hacienda y Crédito Público,Ejecutivo,No Definido,Cerrado,Prestación de servicios,Contratación directa,2019-04-15,2019-02-20,...,1557298.0,66379827,No,Funcionamiento,Distribuido,0,No Definido,0,0,0
9,Distrito Capital de Bogotá,Nacional,agricultura,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2021-01-20,2021-01-21,...,1744200.0,1034967000,No,Inversión,Distribuido,0,Mujer,22032000,0,0
10,Distrito Capital de Bogotá,Nacional,Educación Nacional,Corporación Autónoma,Descentralizada,terminado,Prestación de servicios,Contratación régimen especial,2021-08-05,2021-08-09,...,2640000.0,52800000,No,Funcionamiento,Recursos Propios,0,No Definido,0,0,49500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689321,Distrito Capital de Bogotá,Territorial,Ley de Justicia,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Contratación directa,2022-01-25,2022-02-01,...,7350000.0,24500000,No,Inversión,Recursos Propios,0,No Definido,0,0,24500000
2689327,Huila,Nacional,defensa,Ejecutivo,Descentralizada,terminado,Prestación de servicios,Mínima cuantía,2021-04-06,2021-04-14,...,21000000.0,32550000,No,Funcionamiento,Distribuido,0,No Definido,24500000,0,0
2689331,Distrito Capital de Bogotá,Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,terminado,DecreeLaw092/2017,Contratación régimen especial,2022-03-24,2022-03-29,...,6957491.0,58912135,No,Funcionamiento,Recursos Propios,0,Mujer,0,0,6957491
2689332,Atlántico,Nacional,Trabajo,Ejecutivo,Centralizada,cedido,Prestación de servicios,Contratación directa,2023-01-26,2023-02-01,...,14080000.0,2201918000,No,Inversión,Distribuido,0,No Definido,20680000,0,0


In [7]:
# df_filtradoAnio = df_filtradoAnio.drop_duplicates()
# df_filtradoAnio.info()

In [8]:
df_filtradoAnio.shape

(565610, 31)

In [9]:
df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'
df_filtradoAnio = df_filtradoAnio.drop('Sector', axis=1)

df_filtradoAnio["EsPrestacionServicios"] = df_filtradoAnio['Tipo de Contrato'] == 'Prestación de servicios'
df_filtradoAnio = df_filtradoAnio.drop('Tipo de Contrato', axis=1)

df_filtradoAnio["EsGrupo"] = df_filtradoAnio['Es Grupo'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Grupo', axis=1)

df_filtradoAnio["EsPyme"] = df_filtradoAnio['Es Pyme'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Pyme', axis=1)

df_filtradoAnio["EstaLiquidado"] = df_filtradoAnio['Liquidación'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Liquidación', axis=1)

df_filtradoAnio["EsObligacionAmbiental"] = df_filtradoAnio['Obligación Ambiental'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Obligación Ambiental', axis=1)


df_filtradoAnio["Es PostConflicto"] = df_filtradoAnio['EsPostConflicto'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('EsPostConflicto', axis=1)

df_filtradoAnio["EsRecursosPropios"] = df_filtradoAnio['Origen de los Recursos'] == 'Recursos Propios'
df_filtradoAnio = df_filtradoAnio.drop('Origen de los Recursos', axis=1)


df_filtradoAnio = df_filtradoAnio.drop_duplicates()
df_filtradoAnio.info()

df_filtradoAnio

C:\Users\caste\AppData\Local\Temp\ipykernel_11780\3316525929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'


<class 'pandas.core.frame.DataFrame'>
Index: 498850 entries, 0 to 2689333
Data columns (total 31 columns):
 #   Column                                                              Non-Null Count   Dtype         
---  ------                                                              --------------   -----         
 0   Departamento                                                        498850 non-null  object        
 1   Orden                                                               498850 non-null  object        
 2   Rama                                                                498850 non-null  object        
 3   Entidad Centralizada                                                498850 non-null  object        
 4   Estado Contrato                                                     498850 non-null  object        
 5   Modalidad de Contratacion                                           498850 non-null  object        
 6   Fecha de Firma                                  

,Departamento,Orden,Rama,Entidad Centralizada,Estado Contrato,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,...,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios,EsServicioPublico,EsPrestacionServicios,EsGrupo,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,EsRecursosPropios
0,Santander,Territorial,Ejecutivo,Centralizada,terminado,Contratación directa,2021-08-17,2021-08-19,2021-12-24,NaT,...,13750000,0,True,True,False,False,False,False,False,False
2,Atlántico,Nacional,Ejecutivo,Centralizada,cedido,Contratación directa,2020-02-07,2020-02-07,2020-12-31,2020-02-07,...,0,0,False,True,False,False,False,False,False,False
3,Distrito Capital de Bogotá,Nacional,Ejecutivo,No Definido,Cerrado,Contratación directa,2019-04-15,2019-02-20,2019-12-31,NaT,...,0,0,False,True,False,True,False,False,False,False
9,Distrito Capital de Bogotá,Nacional,Ejecutivo,Descentralizada,terminado,Contratación directa,2021-01-20,2021-01-21,2021-08-31,NaT,...,0,0,False,True,False,False,False,False,False,False
10,Distrito Capital de Bogotá,Nacional,Corporación Autónoma,Descentralizada,terminado,Contratación régimen especial,2021-08-05,2021-08-09,2021-12-31,NaT,...,0,49500000,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689321,Distrito Capital de Bogotá,Territorial,Ejecutivo,Descentralizada,terminado,Contratación directa,2022-01-25,2022-02-01,2023-01-08,NaT,...,0,24500000,False,True,False,False,False,False,False,True
2689327,Huila,Nacional,Ejecutivo,Descentralizada,terminado,Mínima cuantía,2021-04-06,2021-04-14,2021-11-14,NaT,...,0,0,False,True,False,True,False,False,False,False
2689331,Distrito Capital de Bogotá,Territorial,Corporación Autónoma,Descentralizada,terminado,Contratación régimen especial,2022-03-24,2022-03-29,2022-04-30,NaT,...,0,6957491,False,False,False,False,False,False,False,True
2689332,Atlántico,Nacional,Ejecutivo,Centralizada,cedido,Contratación directa,2023-01-26,2023-02-01,2023-11-30,NaT,...,0,0,False,True,False,False,False,False,False,False


In [10]:
# df_filtradoAnio.to_csv('datos_filtrados_20230719.csv', index=False)

In [11]:
muestra_aleatoria = df_filtradoAnio.sample(frac=0.4)
muestra_aleatoria.shape

(199540, 31)

In [12]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(muestra_aleatoria, title="Pandas Profiling Report")
# profile.to_file("secopFiltro2021muestra_profile_20230719_1923.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\caste\anaconda3\envs\workspace\lib\site-packages\ydata_profiling\model\typeset_relations.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return not series.astype("datetime64").isna().all()
C:\Users\caste\anaconda3\envs\workspace\lib\site-packages\ydata_profiling\model\typeset_relations.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return not series.astype("datetime64").isna().all()
C:\Users\caste\anaconda3\envs\workspace\lib\site-packages\ydata_profiling\model\typeset_relations.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return not series.astyp

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
# Aplicar one-hot encoding
df_one_hot = pd.get_dummies(df_filtradoAnio['Estado Contrato'])

# Combinar el DataFrame original con las nuevas columnas
df_final = pd.concat([df_filtradoAnio, df_one_hot], axis=1)

# Eliminar la columna original "Estado Contrato" si deseas
df_final = df_final.drop('Estado Contrato', axis=1)

df_final

,Departamento,Orden,Rama,Entidad Centralizada,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,Fecha de Fin de Ejecucion,...,EsGrupo,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,EsRecursosPropios,Cerrado,Suspendido,cedido,terminado
0,Santander,Territorial,Ejecutivo,Centralizada,Contratación directa,2021-08-17,2021-08-19,2021-12-24,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
2,Atlántico,Nacional,Ejecutivo,Centralizada,Contratación directa,2020-02-07,2020-02-07,2020-12-31,2020-02-07,2020-12-31,...,False,False,False,False,False,False,False,False,True,False
3,Distrito Capital de Bogotá,Nacional,Ejecutivo,No Definido,Contratación directa,2019-04-15,2019-02-20,2019-12-31,NaT,NaT,...,False,True,False,False,False,False,True,False,False,False
9,Distrito Capital de Bogotá,Nacional,Ejecutivo,Descentralizada,Contratación directa,2021-01-20,2021-01-21,2021-08-31,NaT,NaT,...,False,False,False,False,False,False,False,False,False,True
10,Distrito Capital de Bogotá,Nacional,Corporación Autónoma,Descentralizada,Contratación régimen especial,2021-08-05,2021-08-09,2021-12-31,NaT,NaT,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689321,Distrito Capital de Bogotá,Territorial,Ejecutivo,Descentralizada,Contratación directa,2022-01-25,2022-02-01,2023-01-08,NaT,NaT,...,False,False,False,False,False,True,False,False,False,True
2689327,Huila,Nacional,Ejecutivo,Descentralizada,Mínima cuantía,2021-04-06,2021-04-14,2021-11-14,NaT,NaT,...,False,True,False,False,False,False,False,False,False,True
2689331,Distrito Capital de Bogotá,Territorial,Corporación Autónoma,Descentralizada,Contratación régimen especial,2022-03-24,2022-03-29,2022-04-30,NaT,NaT,...,False,False,False,False,False,True,False,False,False,True
2689332,Atlántico,Nacional,Ejecutivo,Centralizada,Contratación directa,2023-01-26,2023-02-01,2023-11-30,NaT,NaT,...,False,False,False,False,False,False,False,False,True,False


In [14]:
estado_contrato = df_final["Cerrado"].copy()
contratos_data = df_final.drop(["terminado","cedido","Suspendido","Cerrado"],axis=1)

In [15]:
# Calculate test and validation set size:
original_count = len(df_final)
training_size = 0.60 # 60% of records
test_size = (1 - training_size) / 2


training_count = int(original_count * training_size)
test_count = int(original_count * test_size)
validation_count = original_count - training_count - test_count

print(training_count, test_count, validation_count, original_count)

299310 99770 99770 498850


In [16]:
from sklearn.model_selection import train_test_split

train_x, rest_x, train_y, rest_y = train_test_split(contratos_data, estado_contrato, train_size=training_count)
test_x, validate_x, test_y, validate_y = train_test_split(rest_x, rest_y, train_size=test_count)

print(len(train_x), len(test_x), len(validate_x))

299310 99770 99770


In [23]:
train_x

,Departamento,Orden,Rama,Entidad Centralizada,Modalidad de Contratacion,Fecha de Firma,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,Fecha de Fin de Ejecucion,...,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos Propios,EsServicioPublico,EsPrestacionServicios,EsGrupo,EsPyme,EstaLiquidado,EsObligacionAmbiental,Es PostConflicto,EsRecursosPropios
535249,Antioquia,Nacional,Corporación Autónoma,Centralizada,Contratación directa,2021-08-06,2021-08-09,2022-01-08,NaT,NaT,...,0,18500000,False,True,False,False,True,False,False,True
1526915,Cesar,Nacional,Ejecutivo,Centralizada,Mínima cuantía,2022-10-03,2022-10-05,2022-10-31,NaT,NaT,...,0,0,False,False,False,True,True,False,False,False
847317,Valle del Cauca,Territorial,Ejecutivo,Centralizada,Contratación directa,2022-01-12,2022-01-14,2022-06-30,NaT,NaT,...,11808000,0,False,True,False,True,False,False,False,False
2002799,Cauca,Territorial,Corporación Autónoma,Descentralizada,Contratación directa,2022-12-02,2022-12-02,2022-12-31,NaT,NaT,...,2300000,0,False,True,False,False,False,False,False,False
510449,Santander,Nacional,Ejecutivo,Centralizada,Contratación directa,2022-08-01,2022-08-01,2022-12-15,NaT,NaT,...,0,12023446,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681147,Antioquia,Territorial,Ejecutivo,Centralizada,Contratación directa,2022-08-03,2022-08-03,2022-12-31,NaT,NaT,...,0,0,True,True,False,False,False,False,False,False
1007402,Risaralda,Territorial,Ejecutivo,Centralizada,Contratación directa,2022-01-17,2022-01-18,2022-07-17,NaT,NaT,...,27240000,0,True,True,False,False,False,False,False,False
1306043,Valle del Cauca,Territorial,Ejecutivo,Centralizada,Contratación directa,2021-06-22,2021-06-24,2021-06-30,NaT,NaT,...,1968000,0,False,True,False,False,False,False,False,False
1651873,Antioquia,Territorial,Ejecutivo,No Definido,Contratación directa,2021-02-01,2021-02-01,2021-06-30,NaT,NaT,...,32500000,0,True,True,False,False,False,False,False,False


In [30]:
train_x.shape

(299310, 30)

# Pipeline

In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import RobustScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

In [78]:
one_hot_encoding = ColumnTransformer([
    (
        'one_hot_encode',
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        [
            "Orden",
            "Entidad Centralizada",
            "Modalidad de Contratacion",
            "Destino Gasto",
            "Género Representante Legal",
            "Departamento",
        ]
    )
])

In [79]:
binarizer = ColumnTransformer([
    (
        'binarizer',
        Binarizer(),
        [
            "EsServicioPublico",
            "EsRecursosPropios",
            "EsGrupo",
            "EsPrestacionServicios",
            "EsPyme",
            "EstaLiquidado",
            "EsObligacionAmbiental",
            "Es PostConflicto",
        ]
    )
])

one_hot_binarized = Pipeline([
    ("binarizer", binarizer),
    ("one_hot_encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
])

In [80]:
scaler = ColumnTransformer([
    ("scaler", RobustScaler(), ["Valor del Contrato","Valor Facturado","Valor Pendiente de Pago","Saldo CDP"])
])

In [81]:
passthrough = ColumnTransformer([
    (
        "passthrough",
        "passthrough",
        [

            "Dias Adicionados",
        ]
    )
])

In [82]:
feature_engineering_pipeline = pipe = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    ("categorical", one_hot_encoding),
                    ("categorical_binarized", one_hot_binarized),
                    ("scaled", scaler),
                    ("pass", passthrough)
                ]
            ),
        )
    ]
)

In [83]:
transformed = feature_engineering_pipeline.fit_transform(train_x)
transformed.shape

(299310, 83)

In [54]:
transformed

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

# Model Trainning

In [84]:
# Get a fresh copy of the pipeline
from sklearn.base import clone

feature_transformer = clone(feature_engineering_pipeline)

features_train_x = feature_transformer.fit_transform(train_x)
features_validate_x = feature_transformer.transform(validate_x)

In [85]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

model = RandomForestClassifier(n_estimators=100)

model.fit(features_train_x, train_y)

RandomForestClassifier()

# Model Validation

In [86]:
from sklearn.metrics import accuracy_score, recall_score

pred_y = model.predict(features_validate_x)

print(accuracy_score(validate_y, pred_y))
print(recall_score(validate_y, pred_y))

0.8666332564899268
0.769608441083191


In [88]:
final_inference_pipeline = Pipeline([
    ("feature_engineering", clone(feature_engineering_pipeline)),
    ("model", RandomForestClassifier(n_estimators=100))
])

In [89]:
final_training_dataset = pd.concat([train_x, validate_x])
final_training_response = pd.concat([train_y, validate_y])

In [90]:
final_inference_pipeline.fit(final_training_dataset, final_training_response)

Pipeline(steps=[('feature_engineering',
                 Pipeline(steps=[('features',
                                  FeatureUnion(transformer_list=[('categorical',
                                                                  ColumnTransformer(transformers=[('one_hot_encode',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse_output=False),
                                                                                                   ['Orden',
                                                                                                    'Entidad '
                                                                                                    'Centralizada',
                                                                                                    'Modalidad '
                                                                                                    'de '
                                                                                                    'Contratacion',
                                                                                                    'Destino '
                                                                                                    'Gasto',
                                                                                                    'Género '
                                                                                                    'Representante '
                                                                                                    'Legal',
                                                                                                    'Departamento'])]...
                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                 sparse_output=False))])),
                                                                 ('scaled',
                                                                  ColumnTransformer(transformers=[('scaler',
                                                                                                   RobustScaler(),
                                                                                                   ['Valor '
                                                                                                    'del '
                                                                                                    'Contrato',
                                                                                                    'Valor '
                                                                                                    'Facturado',
                                                                                                    'Valor '
                                                                                                    'Pendiente '
                                                                                                    'de '
                                                                                                    'Pago',
                                                                                                    'Saldo '
                                                                                                    'CDP'])])),
                                                                 ('pass',
                                                                  ColumnTransformer(transformers=[('passthrough',
                                                                                                   'passthrough',
                                         

In [91]:
test_pred_y = final_inference_pipeline.predict(test_x)

print(accuracy_score(test_pred_y, test_y))
print(recall_score(test_pred_y, test_y))

0.8740202465671043
0.8243667842257134


In [92]:
from joblib import dump

dump(final_inference_pipeline, "Cecop_pipeline.joblib")

['Cecop_pipeline.joblib']